In [3]:
import sys
sys.path.append("/home/saurav/yolov7")


In [4]:
import torch
import numpy as np
from models.experimental import attempt_load
import cv2

/home/saurav/base_env/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
# Load YOLOv7 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = attempt_load("best.pt", map_location=device)  # Load model
model.eval()  # Set to evaluation mode

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


/home/saurav/base_env/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (5): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (6): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (7): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), strid

In [6]:
#importing image 
img = cv2.imread('swornim.jpg',0)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
img = img / 255.0  # Normalize to [0,1] range
img = np.transpose(img, (2, 0, 1))  # Change shape from (H, W, C) to (C, H, W)
img = np.expand_dims(img, axis=0)  # Add batch dimension
img = torch.tensor(img, dtype=torch.float32).to(device)


In [7]:
# Run inference
with torch.no_grad():
    results = model(img)

print(results)


(tensor([[[6.32352e+00, 8.47694e+00, 1.45688e+01,  ..., 1.26344e-02, 2.84241e-02, 6.14424e-03],
         [1.40661e+01, 7.44790e+00, 2.81083e+01,  ..., 1.10872e-02, 1.74151e-02, 4.94524e-03],
         [1.99482e+01, 6.26188e+00, 3.79596e+01,  ..., 1.05794e-02, 1.54647e-02, 4.86624e-03],
         ...,
         [1.95710e+03, 2.01914e+03, 2.15327e+02,  ..., 1.55122e-02, 3.93586e-02, 4.67235e-02],
         [1.98121e+03, 2.02096e+03, 2.25511e+02,  ..., 8.15332e-03, 2.62258e-02, 4.49379e-02],
         [2.02901e+03, 2.03289e+03, 2.25002e+02,  ..., 1.17769e-04, 1.82426e-04, 1.22505e-03]]]), [tensor([[[[[ 5.98094e-01,  1.26455e+00,  2.04400e-01,  ..., -4.35862e+00, -3.53168e+00, -5.08608e+00],
           [ 5.28495e-01,  9.22217e-01,  1.18162e+00,  ..., -4.49081e+00, -4.03285e+00, -5.30437e+00],
           [-1.29418e-02,  5.81303e-01,  2.08345e+00,  ..., -4.53821e+00, -4.15361e+00, -5.32056e+00],
           ...,
           [ 1.26055e-02,  4.95633e-01,  2.37538e+00,  ..., -4.65872e+00, -3.94202e+00

In [8]:
# Post-process the output
results = results[0].cpu().numpy()  # Move to CPU and convert to NumPy
conf_threshold = 0.3  # Confidence threshold for valid detections

print(results.shape)
    

(1, 258048, 18)


In [ ]:
# Draw bounding boxes on the original image
for detection in results:
    x1, y1, x2, y2, conf, cls_id = detection
    if conf > conf_threshold:  # Only consider high-confidence detections
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convert to int
        label = f"Class {int(cls_id)}: {conf:.2f}"  # Label text
        color = (0, 255, 0)  # Green box color

        # Draw rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Show image with detections
cv2.imshow("Detections", img)
cv2.waitKey(0)  # Wait for key press
cv2.destroyAllWindows()

In [22]:
print("result",result)

result (tensor([[[6.32352e+00, 8.47694e+00, 1.45688e+01,  ..., 1.26344e-02, 2.84241e-02, 6.14424e-03],
         [1.40661e+01, 7.44790e+00, 2.81083e+01,  ..., 1.10872e-02, 1.74151e-02, 4.94524e-03],
         [1.99482e+01, 6.26188e+00, 3.79596e+01,  ..., 1.05794e-02, 1.54647e-02, 4.86624e-03],
         ...,
         [1.95710e+03, 2.01914e+03, 2.15327e+02,  ..., 1.55122e-02, 3.93586e-02, 4.67235e-02],
         [1.98121e+03, 2.02096e+03, 2.25511e+02,  ..., 8.15332e-03, 2.62258e-02, 4.49379e-02],
         [2.02901e+03, 2.03289e+03, 2.25002e+02,  ..., 1.17769e-04, 1.82426e-04, 1.22505e-03]]], grad_fn=<CatBackward0>), [tensor([[[[[ 5.98094e-01,  1.26455e+00,  2.04400e-01,  ..., -4.35862e+00, -3.53168e+00, -5.08608e+00],
           [ 5.28495e-01,  9.22217e-01,  1.18162e+00,  ..., -4.49081e+00, -4.03285e+00, -5.30437e+00],
           [-1.29418e-02,  5.81303e-01,  2.08345e+00,  ..., -4.53821e+00, -4.15361e+00, -5.32056e+00],
           ...,
           [ 1.26055e-02,  4.95633e-01,  2.37538e+00,  

In [26]:
print(results.shape)  # Check overall shape
print(results[:5])  # Print first 5 detections


(1, 258048, 18)
[[[     6.3235      8.4769      14.569 ...    0.012634    0.028424   0.0061442]
  [     14.066      7.4479      28.108 ...    0.011087    0.017415   0.0049452]
  [     19.948      6.2619       37.96 ...    0.010579    0.015465   0.0048662]
  ...
  [     1957.1      2019.1      215.33 ...    0.015512    0.039359    0.046724]
  [     1981.2        2021      225.51 ...   0.0081533    0.026226    0.044938]
  [       2029      2032.9         225 ...  0.00011777  0.00018243    0.001225]]]
